In [ ]:
#to connect gooogle drive to the notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#path to the file in google drive
file_path = '/content/drive/My Drive/book.txt'

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 100
#eval_interval = 2500
learning_rate = 3e-4
eval_iters = 25

cpu


In [ ]:
with open(file_path, 'r', encoding = 'utf-8') as file:
  text = file.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']


In [ ]:
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([34, 48, 45,  1, 37, 55, 54, 44, 45, 58, 46, 61, 52,  1, 37, 49, 66, 41,
        58, 44,  1, 55, 46,  1, 29, 66,  0,  0, 42, 65,  1, 26,  8,  1, 20, 58,
        41, 54, 51,  1, 16, 41, 61, 53,  0,  0,  0, 34, 48, 49, 59,  1, 42, 55,
        55, 51,  1, 49, 59,  1, 44, 45, 44, 49, 43, 41, 60, 45, 44,  1, 60, 55,
         1, 53, 65,  1, 47, 55, 55, 44,  1, 46, 58, 49, 45, 54, 44,  1,  3,  1,
        43, 55, 53, 58, 41, 44, 45,  0, 27, 65])


In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)- block_size, (batch_size,))
  #print(ix)
  x = torch.stack([data[i: i+block_size] for i in ix])
  y = torch.stack([data[i+1: i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [ ]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets = None):
    logits = self.token_embedding_table(index)

    if targets is None:
      #print(f'inference : iter : {iter}, logits shape {logits.shape}, logits{logits[:, :3, :]}')
      loss = None
    else:
      B, T , C = logits.shape
      logits = logits.view(B*T, C)
      #print(f'trianing : iter : {iter}, logits shape {logits.shape}, logits{logits[:, :3]}')
      targets = targets.view(B*T)
      #print(f'trianing : iter : {iter}, targets shape {targets.shape}, logits{targets[:3]}')
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self.forward(index)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      index_next = torch.multinomial(probs, num_samples=1)
      index = torch.cat((index, index_next), dim=1)
    return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


EP)g‘y)O!WYlY0.’PcaIowdhPPJBW!Z0—sCHCRQ!LLo“InyP&
!ah?tt0!H—?t0KRWmqSsIM (NXDbM;UzqrCBQLNZXjZ:NtZgXIRQ“B“hBFO!QSqWSytkpnj9)P’!“Ix?Az‘or(‘GOZ
AK0Uy.WRvPjhXOCYI,—Zkl?(‘Q!ghj(hbr‘vsTpzN0OlDen)L?tkZkRF”q”Yfpz-FnOGnA)kmB0UvnSY&VabL”
CBvvskKJQvssF”Y0H&
aX1’wYgGz (beS—K;JQD(F”’.ytkmJ1MYiFE-.
vYBv‘QoRus?tl‘Wj1v“
FJ:yQ!fnlnUu(Wj1R.!a:FVnakcLhxbrqr) jADaL(iXX.m?eI-ftZM’M”qvT()WZrm;,wxD:Rr9,GK.“rs!YHKW1DA“
pterPwDYENHtXoCB!HEGVy&aG!bkIfZx?BQVSLAD
njG!rG0OieR,’P
wYcPdXKer0—VawgMYeRWvS—0)njNSZ
p0Gsx”CijG!D&&


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
for iter in range(max_iters):
  if iter % eval_iters == 0:
    losses = estimate_loss()
    print(f"step: {iter}, train loss: {losses['train']:.3f}, va loss: {losses['val']:.3f}")
  xb, yb = get_batch('train')

  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())

step: 0, train loss: 4.715, va loss: 4.656
step: 25, train loss: 4.711, va loss: 4.681
step: 50, train loss: 4.723, va loss: 4.723
step: 75, train loss: 4.703, va loss: 4.645
4.517852306365967


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


g.?MN?tntUTUuVSpQ!G”Ynr,’H‘yJDLtCNwMY g.jkH?
fcfr!Ac‘—QJctt0bN9UFtMNmuYubeRtdchXVnSGtshbjNZGOcdhtlHuv)—
eD-Wc-AcT1YWWdqk’QiDGFVrm
“’moABDB0KGHYtQhiw&ILkX aamSpIY9) iYDA-VnmH—FbZNwSH?rKEqZGFIlSaEguWy!YELwn—
-J:-’N)ry)OWOdMYZg:“qC;9oSpXqRGW(.’.“;pberUuiT&HF”YU
RdT:a(m)E?s?zAkZD!YcG—(;BQhJ-E’oSZZTPCf(mrPncT;&OTg.ytpeRWASVPmPlKak v t.m‘uVr9rqoqCOZdhBd?lEzrKh,”McdnqS;(v —Sp;FJd:wlEPkBL1aX1S;sAZv(vF”JiUuxDf”’vLqnifs(sO0yak?taCv“E)”?KUQz1VPS:
VRF”?tCNq—W-s;qr1s’”Y:fj
vnb:V&GvY1KJ?KKDnSuNXIj9’UzL?(F ’yt
